In [219]:
# initializing

import pandas as pd
import pandas_gbq as pgbq
from multiprocessing import Pool
from scipy import stats
import os
import time
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from pandas.tools.plotting import scatter_matrix
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline
plt.switch_backend('MacOSX') 

google_project_id = 'project-id'
dataset = 'dataset-id'

In [290]:
test = build_table('message')

message
Starting module 0: edges
Starting module 1: indegree
Starting module 2: nodes
Starting module 3: outdegree
praise_edges table found.
praise_nodes table found.
praise_outdegree table found.
praise_indegree table found.
Joining message modules


KeyError: 'message_edges'

In [289]:
test

,praise_avg_indegree,praise_avg_outdegree,praise_indegree_skew
cid,,,
1,19.998755,13.164265,0.519170
10,24.666667,1.013699,23.333333
100,22.278788,1.145528,18.448485
1000,13.949219,1.441082,8.679687
1001,12.631799,1.482081,7.523013
1002,12.448052,1.307640,8.519481
1003,14.727599,1.656187,7.892473
1004,11.245902,1.345978,7.355191
1005,17.085859,1.629576,9.484848


In [271]:
def build_table(module_type):
    print(module_type)
    if __name__ == '__main__':
        with Pool(processes=len(sql_modules)) as pool:
            result = pool.map(bigquery_worker, (range(len(sql_modules),)))
            pool.close()
            pool.join()
    joined = result[0].set_index('cid', drop=True)
    print('Joining ',module_type,' modules', sep='')
    for i in range(len(result)-1):
        joined = joined.join(result[i+1].set_index('cid'), how='left', lsuffix='_left')
    table = joined.replace([np.inf,-np.inf], np.nan)
    table.fillna(0, inplace=True)
    table.sort_index(inplace=True)
    if re.match('community_facts', module_type) is None:
        table = add_network_stats(table, module_type)
    #pgbq.to_gbq(network_stats, dataset+'.'+sql_module_type+'_stats', google_project_id, if_exists='replace', verbose=False)
    print('Done', sep='')
    return(table)

def open_active_modules(module_type):
    sql_files = list(map(lambda x: os.path.splitext(x), os.listdir(directory)))
    sql_modules = list()
    for i in range(len(sql_files)):
        if sql_files[i][1] == '.on':
            sql_modules.append(sql_files[i][0])
    return(sql_modules)

def bigquery_worker(i):
    sql_module = sql_modules[i]
    print('Starting module ',i,': ',sql_modules[i], sep='')
    try:
        result = pgbq.read_gbq('select * from community_networks.'+module_type+'_'+sql_module, google_project_id, dialect='standard', verbose=False)
        print(module_type,'_',sql_module,' table found.', sep='')
    except:
        print(module_type,'_',sql_module,' table does not exist, generating now', sep='')
        with open(directory+'/'+sql_module+'.on') as query_file:
            query = query_file.read()
        result = pgbq.read_gbq(query, google_project_id, dialect='standard', verbose=False)
        print('Creating new ',module_type,'_',sql_module,' table', sep='')
        pgbq.to_gbq(result, dataset+'.'+module_type+'_'+sql_module, google_project_id, if_exists='replace', verbose=False)
        print('Module ',i,': ',sql_module,' query complete', sep='')
    return (result)

def add_network_stats(df, sql_module_type):
    edges = df[sql_module_type+'_edges']
    nodes = df[sql_module_type+'_nodes']
    avg_indegree = df[sql_module_type+'_avg_indegree']
    avg_outdegree = df[sql_module_type+'_avg_outdegree']
    #df[sql_module_type+'_network_density_X_100'] = (edges/(nodes*(nodes-1)/2))*100
    df[sql_module_type+'_indegree_skew'] = (avg_indegree-avg_outdegree)/avg_outdegree
    df.drop([sql_module_type+'_edges',sql_module_type+'_nodes'], axis=1, inplace=True)
    return(df)

In [ ]:
sql_module_types = ['community_facts','transaction','follow','message','praise']

data = pgbq.read_gbq('select id cid from vs_reporting.communities', google_project_id, dialect='standard', verbose=False).set_index('cid', drop=False)

for module_type in sql_module_types:
    directory = '/Users/joshharris/community_health/sql/module_types/'+module_type
    sql_modules = open_active_modules(module_type)
    module_table = build_table(module_type)
    data = data.join(module_table, how='left', lsuffix='_left')

labeled_features = data.replace([np.inf,-np.inf], np.nan)
labeled_features.fillna(0, inplace=True)
labeled_features.drop(labeled_features[labeled_features.avg_communities_per_user == 0].index, inplace=True)
labeled_features.drop(labeled_features[labeled_features.items_posted_last_month < 100].index, inplace=True)
#labeled_features.drop(labeled_features[labeled_features.items_posted_last_month > 100000].index, inplace=True)


community_facts
Starting module 9: pct_mau_top_members
Starting module 8: pct_mau_recd_msg_from_admin_last_month
Starting module 2: distance_from_center
Starting module 0: _community_age
Starting module 1: communities_per_user
Starting module 3: items_posted
Starting module 4: list_price_variance
Starting module 5: m3_total_retention
Starting module 6: market_activation_and_items_per_capita
Starting module 7: pct_female_mau
Starting module 11: pct_with_about_me
Starting module 10: pct_memberships_revoked
Starting module 12: sale_price_variance
community_facts_pct_with_about_me table found.
community_facts_items_posted table found.
community_facts_distance_from_center table found.
community_facts_pct_female_mau table found.
community_facts_m3_total_retention table found.
community_facts_communities_per_user table found.
community_facts__community_age table found.
community_facts_market_activation_and_items_per_capita table found.
community_facts_pct_mau_top_members table found.
communit

In [286]:
labeled_features

,cid,name,age,avg_communities_per_user,avg_distance_from_center,items_posted_last_month,list_price_variance,m3_total_retention,adult_population,last_month_mau_activation_pct,...,follow_avg_indegree,follow_avg_outdegree,follow_indegree_skew,message_avg_weight,message_avg_indegree,message_avg_outdegree,message_indegree_skew,praise_avg_indegree,praise_avg_outdegree,praise_indegree_skew
cid,,,,,,,,,,,,,,,,,,,,,
33,33,Quad Cities Iowa/Illinois,1656.0,8.631293,35.245926,332.0,3.507207e+07,0.037037,0.0,0.000000,...,18.222543,1.117908,15.300578,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232,232,"Hurley, MS Buy & Sell",1407.0,4.193581,32.502186,312.0,4.784763e+14,0.096774,0.0,0.000000,...,11.863845,1.737880,5.826619,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1707,1707,"Honolulu, HI Buy and Sell",570.0,35.360465,14.707712,141.0,1.109297e+11,0.051282,0.0,0.000000,...,27.245902,1.004837,26.114754,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1832,1832,"Newark, NJ Buy and Sell",570.0,30.847134,11.010131,126.0,7.794790e+06,0.162791,0.0,0.000000,...,47.139241,1.004315,45.936709,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1834,1834,"Clifton, NJ Buy and Sell",570.0,30.028986,12.111781,172.0,2.047857e+06,0.058824,0.0,0.000000,...,3.764706,1.054945,2.568627,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1835,1835,"New Brunswick, NJ Buy and Sell",570.0,24.573099,18.104309,140.0,2.073673e+13,0.062500,0.0,0.000000,...,9.512500,1.031165,8.225000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1887,1887,"Columbia, SC Buy and Sell",570.0,10.174515,33.103656,122.0,2.766625e+09,0.060241,0.0,0.000000,...,22.728682,1.367112,15.625323,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,28,Upstate South Carolina,1668.0,3.615519,29.257130,408.0,2.393008e+11,0.131944,0.0,0.000000,...,15.535714,1.141304,12.612245,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,65,"Williston Connections, ND",1549.0,3.031809,19.118557,1588.0,4.226847e+15,0.263158,0.0,0.000000,...,11.224092,1.476751,6.600531,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [280]:
# Cross correlate community facts and output key correlations

variables = list(labeled_features)[2:len(labeled_features)] #this is not a good way to create the list of variables.. why not
print(variables)
corr_matrix = pd.DataFrame(index=variables, columns=variables)
key_correlations = pd.DataFrame(columns=['pair','coefficient'])
corr_checked = list()

i = 0
j = 0

for i in range(len(variables)):
    for j in range(len(variables)):
        pair = variables[i]+','+variables[j]
        rho, pval = stats.spearmanr(labeled_features[[variables[i],variables[j]]])
        corr_matrix.set_value(variables[i], variables[j], rho)
        if pair not in corr_checked and rho < 0.99 and (rho > 0.4 or rho < -0.4):
            key_correlations.loc[len(key_correlations)] = [pair, rho]
            corr_checked.append(variables[j]+','+variables[i]) 
            

corr_matrix.reset_index(inplace=True)
key_correlations.sort_values(by='coefficient', ascending=False, inplace=True)
key_correlations.reset_index(drop=True, inplace=True)

#Save to BigQuery
#pgbq.to_gbq(transaction_corr_matrix, 'community_networks.transaction_corr_matrix_over_1000', google_project_id, if_exists='replace')
#pgbq.to_gbq(transaction_key_correlations, 'community_networks.transaction_key_correlations_over_1000', google_project_id, if_exists='replace')


['age', 'avg_communities_per_user', 'avg_distance_from_center', 'items_posted_last_month', 'list_price_variance', 'm3_total_retention', 'adult_population', 'last_month_mau_activation_pct', 'items_per_capita', 'pct_female_mau', 'conversations_last_month', 'pct_mau_top_members', 'pct_memberships_revoked', 'pct_with_about_me', 'sale_price_variance', 'transaction_avg_weight', 'transaction_avg_indegree', 'transaction_avg_outdegree', 'transaction_indegree_skew', 'follow_avg_indegree', 'follow_avg_outdegree', 'follow_indegree_skew']


In [281]:
key_correlations

,pair,coefficient
0,"adult_population,items_per_capita",0.959829
1,"adult_population,last_month_mau_activation_pct",0.955260
2,"transaction_avg_outdegree,follow_avg_outdegree",0.940031
3,"pct_mau_top_members,transaction_avg_outdegree",0.793989
4,"pct_mau_top_members,follow_avg_outdegree",0.758816
5,"items_posted_last_month,follow_avg_outdegree",0.730256
6,"items_posted_last_month,pct_mau_top_members",0.700357
7,"items_posted_last_month,transaction_avg_outdegree",0.699023
8,"transaction_indegree_skew,follow_indegree_skew",0.692379
9,"avg_communities_per_user,transaction_indegree_...",0.667225


In [282]:
corr_matrix

,index,age,avg_communities_per_user,avg_distance_from_center,items_posted_last_month,list_price_variance,m3_total_retention,adult_population,last_month_mau_activation_pct,items_per_capita,...,pct_memberships_revoked,pct_with_about_me,sale_price_variance,transaction_avg_weight,transaction_avg_indegree,transaction_avg_outdegree,transaction_indegree_skew,follow_avg_indegree,follow_avg_outdegree,follow_indegree_skew
0,age,1,-0.595342,0.233421,0.449728,0.413884,0.140649,-0.214844,-0.152668,-0.153474,...,0.559064,0.356774,0.390081,0.103052,0.0373006,0.377555,-0.389745,0.207349,0.339352,-0.0816445
1,avg_communities_per_user,-0.595342,1,-0.0325929,-0.691052,-0.403914,-0.410375,0.09777,0.0435885,0.0412715,...,-0.52094,-0.259875,-0.51824,0.0773403,-0.0542556,-0.64109,0.667225,0.0442324,-0.647879,0.479621
2,avg_distance_from_center,0.233421,-0.0325929,1,-0.177862,0.0364129,-0.0997348,-0.187911,-0.0972148,-0.105256,...,-0.0919674,0.0573853,-0.010381,0.188126,0.154904,0.0531774,0.0709907,0.279248,0.0446637,0.215971
3,items_posted_last_month,0.449728,-0.691052,-0.177862,1,0.50085,0.602079,0.168396,0.164558,0.17635,...,0.388048,0.286741,0.65648,-0.173203,0.0608127,0.699023,-0.722785,-0.11772,0.730256,-0.599795
4,list_price_variance,0.413884,-0.403914,0.0364129,0.50085,1,0.287969,0.108919,0.10362,0.105845,...,0.272963,0.106008,0.536557,-0.0809878,-0.0538495,0.311977,-0.361424,-0.0377555,0.345271,-0.250422
5,m3_total_retention,0.140649,-0.410375,-0.0997348,0.602079,0.287969,1,0.362147,0.399077,0.401985,...,0.199473,0.0832816,0.456585,-0.350402,0.113914,0.511852,-0.477964,-0.123587,0.551984,-0.462545
6,adult_population,-0.214844,0.09777,-0.187911,0.168396,0.108919,0.362147,1,0.95526,0.959829,...,-0.244994,-0.273342,0.140009,-0.523873,0.0870298,-0.112897,0.203833,-0.0760307,0.0261646,-0.0534503
7,last_month_mau_activation_pct,-0.152668,0.0435885,-0.0972148,0.164558,0.10362,0.399077,0.95526,1,0.99802,...,-0.177785,-0.256592,0.159566,-0.522072,0.105977,-0.079972,0.177264,-0.0597124,0.0375777,-0.0473179
8,items_per_capita,-0.153474,0.0412715,-0.105256,0.17635,0.105845,0.401985,0.959829,0.99802,1,...,-0.177469,-0.24471,0.16306,-0.524263,0.103659,-0.0716626,0.167215,-0.0638555,0.0476461,-0.0589164
9,pct_female_mau,0.38243,-0.307842,0.0480203,0.307588,0.226002,0.188613,0.0488722,0.0840815,0.08023,...,0.310932,0.274198,0.218485,0.0883482,0.168367,0.32723,-0.249887,0.166527,0.298949,-0.0476554


## Notes

pct_with_about_me correlates with pct_female_mau
 - action: encourage men to fill in their about me
 


In [ ]:
scaler = StandardScaler()

features = pd.concat([pd.DataFrame(labeled_features.iloc[:,3:])], axis = 1, join_axes = [labeled_features.index])
scaled_data = pd.DataFrame(scaler.fit_transform(features))
data3D = pd.DataFrame(PCA(n_components=3).fit_transform(scaled_data)) # Reduce dimensions from 4 to 3 for visualization
data2D = pd.DataFrame(PCA(n_components=2).fit_transform(scaled_data)) # Reduce dimensions from 4 to 2 for visualization

# run kmeans with 10 sets of clusters seeds
# extract classification labels and cluster centers 
# do some prep for vizualization
def cluster_it_up(N):
    
    kmeans = KMeans(init = 'k-means++', n_clusters = N, n_init = 100).fit(scaled_data) # it's as easy as a function call
    labels = pd.DataFrame((kmeans.labels_)) # extract labels
    centers = pd.DataFrame(kmeans.cluster_centers_) # extract centers
    centers.columns = features.columns # give centers table readable column names
    labels.columns = ['label'] 
    clustered_features = pd.concat([labeled_features, labels], axis=1, join_axes = [labeled_features.index]) # make DataFrame that is labeled users and their features
    viz3D = pd.concat([data3D, labels], axis = 1, join_axes = [data3D.index]) 
    viz2D = pd.concat([data2D, labels], axis = 1, join_axes = [data2D.index])
    return clust_out, centers, N, labeled_features, labels, viz3D, viz2D

In [ ]:
clustered_features, centers, N, labeled_features, labels, viz3D, viz2D  = cluster_it_up(18)

In [ ]:
%matplotlib inline
colors = plt.cm.rainbow(np.linspace(0, 1, len(centers)))

#2D viz
fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
for i,c in enumerate(colors):
    single_clus = viz2D.loc[viz2D['label'] == i]
    plt.scatter(single_clus[0], single_clus[1], s=3, c = c, label=str(i))
plt.legend()
plt.show()

#3D viz
fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
ax = fig.add_subplot(111, projection='3d')
for i,c in enumerate(colors):
    single_clus = viz3D.loc[viz3D['label'] == i]
    ax.scatter(single_clus[1], single_clus[2], single_clus[0], s=20, c = c, label=str(i))
plt.legend()
plt.show()

#scatter matrix
#scatter_matrix(data, alpha=0.1, figsize=(6, 6), diagonal='kde')
#plt.show()

corrected_centers = pd.DataFrame(scaler.inverse_transform(centers))
corrected_centers.columns = list(features)
corrected_centers.sort_values(by='m3_total_retention', inplace=True)
corrected_centers

In [ ]:
corrected_centers.to_csv('/Users/joshharris/Google Drive/Community Health/ ')

In [244]:
clust_out[clust_out['label'] == 3]


,cid,name,age,avg_communities_per_user,avg_distance_from_center,m3_total_retention,pct_female_mau,pct_with_about_me,transaction_avg_weight,transaction_avg_indegree,transaction_avg_outdegree,transaction_indegree_skew,follow_avg_indegree,follow_avg_outdegree,follow_indegree_skew,label
cid,,,,,,,,,,,,,,,,
2481,2481,"Summerside, Prince Edward Island Buy and Sell",266.0,51.782609,19.779159,0.062992,0.680000,0.026667,1.618952,42.263158,1.645492,24.684211,16.366667,1.020790,15.033333,3.0
244,244,"New Castle County, DE Buy Sell & Trade",1388.0,8.104987,24.452817,0.086957,0.728261,0.021739,3.408163,75.562738,6.462764,10.692015,20.131488,1.192946,15.875433,3.0
2060,2060,"Biloxi, MS Buy and Sell",352.0,10.391026,40.964931,0.000000,0.714286,0.047619,1.879846,39.685637,3.010691,12.181572,11.247748,1.191317,8.441441,3.0
2090,2090,"Florence, SC Buy and Sell",352.0,14.064815,39.775283,0.000000,0.666667,0.027778,2.381997,34.795580,5.924741,4.872928,22.566667,1.250231,17.050000,3.0
2580,2580,"Fort Smith, Northwest Territories Buy and Sell",223.0,187.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,2.000000,3.0
243,243,The Mama Swap of Maine,1388.0,2.868576,28.679810,0.230769,0.988943,0.088452,2.439362,26.508811,12.284791,1.157856,10.755054,1.976617,4.441141,3.0
2512,2512,"Wandsworth and Battersea, UK Buy and Sell",266.0,128.555556,10.268915,0.068966,0.675676,0.054054,1.666667,4.000000,1.666667,1.400000,6.285714,1.023256,5.142857,3.0
2258,2258,"Drumheller, Alberta Buy and Sell",266.0,18.180556,47.983966,0.111111,0.837209,0.023256,1.384260,40.642857,2.168445,17.742857,6.236842,1.102326,4.657895,3.0
2259,2259,"Eckville, Alberta Buy and Sell",266.0,43.357143,37.283464,0.000000,0.923077,0.076923,1.581215,29.204082,1.711722,16.061224,10.115385,1.023346,8.884615,3.0
